In [1]:
from pybaseball import team_batting, team_pitching

In [2]:
## Idea 1: WAR (Batting + Pitching), and correlation with W/L percentage and success
## List top 10 luckiest teams, top 10 unluckiest teams, per year, last decade, last two decades, and in history.

WarTotalDict = dict['oWAR': float, 'pWAR': float, 'totalWAR': float]

teamStatsBatting  = team_batting(2024)
teamStatsPitching = team_pitching(2024)
allTeams : list[str] = teamStatsBatting['Team'].values

teamAgg : list[WarTotalDict] = []

for team in allTeams:
  batWar = teamStatsBatting[teamStatsBatting['Team'] == team]['WAR'].values[0]
  pitchWar = teamStatsPitching[teamStatsPitching['Team'] == team]['WAR'].values[0]
  winsCount = teamStatsPitching[teamStatsPitching['Team'] == team]['W'].values[0]
  totalWar = batWar + pitchWar
  warTotal : WarTotalDict = {"team": team, "oWAR": batWar, "pWAR": pitchWar, "totalWAR": batWar + pitchWar, "xWins": totalWar + 47.7, "actualWins": winsCount}
  teamAgg.append(warTotal)

a = sorted(teamAgg, key = lambda a: a["totalWAR"], reverse=True)

# team_batting_dodgers = teamStats[teamStats['Team'] == 'LAD']

In [3]:
## Idea 2: Biggest playoff losers - If we assume the team that beat a given team is better, what is the ordering of best to worst?
"""
Example: Playoff Ranking 2024

WC: Tigers over Astros (2)
WC: Royals over Orioles (2)
WC: Guardians over N/A (2)
WC: Yankees over N/A (2)
DS: Guardians over Tigers (5)
DS: Yankees over Royals (4)
CS: Yankees over Guardians (5)

WC: Mets over Brewers (3)
WC: Padres over Braves (2)
WC: Dodgers over N/A (2)
WS: Phillies over N/A (2)
DS: Dodgers over Padres (5)
DS: Mets over Phillies (5)
CS: Dodgers over Mets (6)
WS: Dodgers over Yankees  (5)


Ranking:
1. Dodgers
2. Yankees (Dodgers 1)
3. Mets (Dodgers 1)
4. Guardians (Yankees 2)
5. Padres (Dodgers 1)
6. Royals (Yankees 2)
7. Phillies ( Mets 3 -> Dodgers 1)
8. Tigers ( Guardians 4 -> Yankees 2)
9. Brewers ( Mets 3 -> Dodgers 1)
10. Braves (Padres 5 -> Dodgers 1)
11. Orioles (Royals 6 -> Yankees 2 -> Dodgers 1)
12. Astros (Tigers 8 -> Guardians 4 -> Yankees 2)
"""

from pybaseball import schedule_and_record, retrosheet

ws = retrosheet.world_series_logs()
cs = retrosheet.lcs_logs()
ds = retrosheet.division_series_logs()
wc = retrosheet.wild_card_logs()
dodgersRecord = schedule_and_record(2024, "LAD")



http://www.baseball-reference.com/teams/LAD/2024-schedule-scores.shtml


/Users/jonahdf/local-documents/baseball-playground/baseball-env/lib/python3.12/site-packages/pybaseball/team_results.py:75: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Attendance'].replace(r'^Unknown$', np.nan, regex=True, inplace = True) # make this a NaN so the column can benumeric


In [4]:
## List Team names from retrosheet
teamsSet: set = set()
for index, row in ws.iterrows():
  teamsSet.add(row["visiting_team"])
  teamsSet.add(row["home_team"])

for index, row in cs.iterrows():
  teamsSet.add(row["visiting_team"])
  teamsSet.add(row["home_team"])

for index, row in ds.iterrows():
  teamsSet.add(row["visiting_team"])
  teamsSet.add(row["home_team"])

for index, row in wc.iterrows():
  teamsSet.add(row["visiting_team"])
  teamsSet.add(row["home_team"])
print(sorted(teamsSet))



['ANA', 'ARI', 'ATL', 'BAL', 'BOS', 'BRO', 'BSN', 'CAL', 'CHA', 'CHN', 'CIN', 'CLE', 'COL', 'DET', 'FLO', 'HOU', 'KCA', 'LAN', 'MIA', 'MIL', 'MIN', 'MLN', 'MON', 'NY1', 'NYA', 'NYN', 'OAK', 'PHA', 'PHI', 'PIT', 'SDN', 'SEA', 'SFN', 'SLA', 'SLN', 'TBA', 'TEX', 'TOR', 'WAS', 'WS1']


In [5]:
## Map of RetroSheet team names to current team names:
teamNamesToCurrent: dict[str] = {
    "ANA": "LAA",
    "ARI": "ARI",
    "ATL": "ATL",
    "BAL": "BAL",
    "BOS": "BOS",
    "BRO": "LAD",
    "BSN": "ATL",
    "CAL": "LAA",
    "CHA": "CHW",
    "CHN": "CHC",
    "CIN": "CIN",
    "CLE": "CLE",
    "COL": "COL",
    "DET": "DET",
    "FLO": "MIA",
    "HOU": "HOU",
    "KC1": "OAK",
    "KCA": "KCR",
    "LAN": "LAD",
    "MIA": "MIA",
    "MIL": "MIL",
    "MIN": "MIN",
    "MLN": "ATL",
    "MON": "WSN",
    "NY1": "SFG",
    "NYA": "NYY",
    "NYN": "NYM",
    "OAK": "OAK",
    "PHA": "OAK",
    "PHI": "PHI",
    "PIT": "PIT",
    "SDN": "SDP",
    "SEA": "SEA",
    "SFN": "SFG",
    "SLA": "STL",
    "SLN": "STL",
    "TBA": "TBR",
    "TEX": "TEX",
    "TOR": "TOR",
    "WAS": "WAS",
    "WS1": "MIN",
}

In [7]:
from pandas import DataFrame

SeriesWinner = dict["year": int, "round": str, "winner": str, "loser": str, "games": int]
OngoingSeries = dict["gamesCount": int, "lastGameWinner": str, "lastGameLoser": str]


# Summarizes series results for a given round in a given year
def get_series_winners_and_games_by_year(year: int, round: str, data: DataFrame) -> list[SeriesWinner]:
  data["date"] = data["date"].apply(str)
  filtered_year = data[data["date"].str[:4] == str(year)]
  
  seriesResults : list[SeriesWinner] = []
  series: dict[OngoingSeries] = {}

  # Construct Ongoing Series Objects
  for _, row in filtered_year.iterrows():
    teamsKey = str(sorted([row["visiting_team"], row["home_team"]]))
    lastGameWinner = row["visiting_team"] if row["visiting_score"] > row["home_score"] else row["home_team"]
    lastGameLoser = row["visiting_team"] if row["visiting_score"] < row["home_score"] else row["home_team"]
    gamesCount = row["home_team_game_num"]
    series[teamsKey] = { "gamesCount": gamesCount, "lastGameWinner": lastGameWinner, "lastGameLoser": lastGameLoser }

  # Construct Series Results Objects
  for _, item in series.items():
    seriesResults.append(
      {
        "year": year,
        "round": round,
        "winner": teamNamesToCurrent[item["lastGameWinner"]],
        "loser": teamNamesToCurrent[item["lastGameLoser"]],
        "games": item["gamesCount"],
      }
    )
  return seriesResults

# Gets all round results by year
def get_all_rounds_by_year(year: int, wc_data = wc, ds_data = ds, cs_data = cs, ws_data = ws):
    return (
        get_series_winners_and_games_by_year(year, "wc", wc_data)
        + get_series_winners_and_games_by_year(year, "ds", ds_data)
        + get_series_winners_and_games_by_year(year, "cs", cs_data)
        + get_series_winners_and_games_by_year(year, "ws", ws_data)
    )

# construct large data set of all years
all_playoffs_ever: list[SeriesWinner] = []
for year in range(1900, 2024):
   all_playoffs_ever.extend(get_all_rounds_by_year(year))

df = DataFrame(all_playoffs_ever)
df.to_csv('playoff-data.csv')


In [8]:
import pandas as pd
all_playoffs_ever = pd.read_csv('playoff-data.csv', index_col=0)

"""
Given a year of playoffs, sort by winners and losers.

Ordering is defined by the following algorithm:
1. WS winners and losers are 1 and 2 respectively
2. CS losers are 3 and 4, ordered by whether they lost to the eventual WS winner
3. DS losers are ordered by the ranking of who they lost to
4. WC losers are ordered by the ranking of who they lost to
"""
def sort_winners(year: int, data = all_playoffs_ever):
  df = data[data["year"] == year]
  if (df.empty):
    print("No data exists for year", year)
    return {}
  ranking = {}
  for round in ["ws", "cs", "ds", "wc"]:
    round_results = df[df["round"] == round]
    within_round_ranking = []
    if round == "ws":
      # print("winner", round_results["winner"].values[0])
      ranking[round_results["winner"].values[0]] = 1
      ranking[round_results["loser"].values[0]] = 2
    else:
      for _, round_result in round_results.iterrows():

        loser = round_result["loser"]
        winner = round_result["winner"]
        winner_ranking = ranking[round_result["winner"]]
        # print(f"Trying to find ranking for {loser}, who lost to {winner}, ranked {winner_ranking}")
        
        # Insert in appropriate place in list
        insertion_index = 0
        for i, team in enumerate(within_round_ranking):
          # print("comparing with loser result for ", team)
          within_round_competitor_winner = round_results[round_results["loser"] == team]["winner"].values[0]
          if winner_ranking < ranking[within_round_competitor_winner]:
            # print("setting insertion index to ", i)
            insertion_index = i
            break
          # print("iterating insertion index")
          insertion_index += 1
        within_round_ranking.insert(insertion_index, loser)
      for loser in within_round_ranking:
        # print(f"loser {loser} is ranking {len(ranking.values()) + 1}")
        ranking[loser] = len(ranking.values()) + 1
  return ranking

all_rankings = []
for year in range(1914, 2024):
  all_rankings.append(sort_winners(year))

all_team_stats = {}
for team in teamNamesToCurrent.values():
  team_stats = {"numAppearances": 0, "placements": [], "averagePlacement": "na"}
  for ranking in all_rankings:
    if team in ranking:
      team_stats["numAppearances"]+= 1
      team_stats["placements"].append(ranking[team])
      team_stats["averagePlacement"] = sum(team_stats["placements"]) / team_stats["numAppearances"]
  all_team_stats[team] = team_stats

df = DataFrame(all_team_stats).transpose()
    

No data exists for year 1994


In [ ]:
## Idea: Horserace. Take a trailing n-game window, visualize a horserace over the years of teams rising and falling in terms of ranked winning percentage. 
## https://pypi.org/project/bar-chart-race/

In [ ]:
a = []
a.append({"b": "asd"})
print(a)